In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# Adicionar TimeSeriesSplit e mean_absolute_error, etc. diretamente se necessário fora da função
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping # Importar EarlyStopping
import yfinance as yf

# Função carregar_dados_yahoo (sem alterações)
def carregar_dados_yahoo(tickers, start_date_val, end_date_val, start_date_test, end_date_test, salvar_csv=False, pasta_csv="dados_acoes_csv"):
    # ... (código original sem alterações) ...
    dados = {}
    if salvar_csv:
        os.makedirs(pasta_csv, exist_ok=True)
    for ticker in tickers:
        print(f"Baixando dados para {ticker}...")
        # Baixar dados de validação (treino + validação interna do tuner/CV)
        df_val_train = yf.download(ticker, start=start_date_val, end=end_date_val)
        # Baixar dados de teste final (hold-out)
        df_test_final = yf.download(ticker, start=start_date_test, end=end_date_test)

        if df_val_train.empty or df_test_final.empty:
            print(f"Aviso: Nenhum dado encontrado para {ticker} (val ou test). Pulando ticker.")
            continue

        # Usar apenas 'Close'
        df_val_train = df_val_train[['Close']].rename(columns={'Close': 'Preço'})
        df_test_final = df_test_final[['Close']].rename(columns={'Close': 'Preço'})

        # ATENÇÃO: Armazenar separadamente para clareza na função de experimento
        dados[ticker] = {'val_train': df_val_train, 'test_final': df_test_final}

        if salvar_csv:
            # Salvar concatenado se precisar do arquivo completo, ou salvar separado
            caminho_csv_val = os.path.join(pasta_csv, f"{ticker}_val_train_{start_date_val}_to_{end_date_val}.csv")
            caminho_csv_test = os.path.join(pasta_csv, f"{ticker}_test_{start_date_test}_to_{end_date_test}.csv")
            df_val_train.to_csv(caminho_csv_val)
            df_test_final.to_csv(caminho_csv_test)
            print(f"Dados de Val/Train de {ticker} salvos em {caminho_csv_val}.")
            print(f"Dados de Teste Final de {ticker} salvos em {caminho_csv_test}.")
    return dados


# Função criar_janelas (sem alterações)
def criar_janelas(dados, janela):
    # ... (código original sem alterações) ...
    X, y = [], []
    for i in range(len(dados) - janela):
        X.append(dados[i:i + janela])
        y.append(dados[i + janela])
    return np.array(X), np.array(y)

# Função build_model (sem alterações)
def build_model(hp, input_shape):
    # ... (código original sem alterações) ...
    model = keras.Sequential()
    # Adicionar camadas LSTM
    for i in range(hp.Int('num_lstm_layers', 1, 3)): # Exemplo: 1 a 3 camadas
        return_sequences = i < hp.Int('num_lstm_layers', 1, 3) - 1
        model.add(layers.LSTM(
            units=hp.Int(f'units_lstm_{i}', min_value=32, max_value=128, step=32), # Ajustar range se necessário
            return_sequences=return_sequences,
            input_shape=input_shape if i == 0 else None
        ))
        model.add(layers.Dropout(hp.Float(f'dropout_lstm_{i}', 0.1, 0.3, step=0.1))) # Dropout moderado

    # Adicionar camadas Densas (opcional, mas comum antes da saída)
    for i in range(hp.Int('num_dense_layers', 0, 2)): # Exemplo: 0 a 2 camadas
         if hp.Int('num_dense_layers', 0, 2) > 0: # Só adiciona se num_dense_layers > 0
            model.add(layers.Dense(
                units=hp.Int(f'units_dense_{i}', min_value=16, max_value=64, step=16), # Menos unidades talvez
                activation='relu'
            ))
            model.add(layers.Dropout(hp.Float(f'dropout_dense_{i}', 0.1, 0.3, step=0.1)))

    # Camada de saída
    model.add(layers.Dense(1)) # Saída única (preço)

    # Compilação
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 5e-4])), # Menos opções talvez
        loss='mean_squared_error'
    )
    return model


# Configurações (ajustar se necessário)
tickers = [ "BEEF3.SA", "PETR4.SA", "SOJA3.SA", "GGBR3.SA", "CSNA3.SA", "VALE3.SA", "JBSS3.SA", "BRFS3.SA", "SUZB3.SA"]
start_date_val = "2020-01-01"
end_date_val = "2022-12-31" # Dados para treino e validação CV
start_date_test = "2023-01-01"
end_date_test = "2023-12-31" # Dados para teste final (hold-out)
janela_min = 1 # Aumentar janela mínima
janela_max = 2 # Aumentar janela máxima
n_splits_cv = 5  # Número de folds para TimeSeriesSplit
max_trials_tuner = 10 # Aumentar tentativas do tuner
epochs_tuner = 50    # Epochs para busca do tuner
epochs_final = 100   # Epochs para treino final (com early stopping)
resultados_dir = "resultados_lstm_cv"
os.makedirs(resultados_dir, exist_ok=True)

# Carregar os dados (agora retorna um dicionário com 'val_train' e 'test_final')
dados_por_ticker = carregar_dados_yahoo(tickers, start_date_val, end_date_val, start_date_test, end_date_test, salvar_csv=True, pasta_csv="dados_acoes_csv_cv")

# --- Função de Experimento Específico MODIFICADA ---
def rodar_experimento_especifico(ticker, janela, dados_ticker):
    print(f"\n--- Iniciando Experimento: {ticker}, Janela {janela} ---")

    # Definir nomes de arquivos
    base_filename = f"{resultados_dir}/{ticker}_Janela_{janela}"
    metrics_path = f"{base_filename}_metrics.csv"
    hiperparametros_path = f"{base_filename}_hiperparametros.csv"
    grafico_path = f"{base_filename}_grafico_teste_final.png"
    previsoes_path = f"{base_filename}_previsoes_teste_final.csv"

    if os.path.exists(metrics_path):
        print(f"Resultados já existem para {ticker}, Janela {janela}. Pulando...")
        return

    # 1. Preparar Dados de Treino/Validação (2020-2022)
    df_val_train = dados_ticker['val_train'].copy()
    if df_val_train.isnull().any().any():
        print(f"Aviso: Dados nulos encontrados em df_val_train para {ticker}. Preenchendo com ffill/bfill.")
        df_val_train.fillna(method='ffill', inplace=True)
        df_val_train.fillna(method='bfill', inplace=True) # Para garantir preenchimento no início

    scaler = MinMaxScaler()
    dados_val_train_scaled = scaler.fit_transform(df_val_train)
    X_val_train_full, y_val_train_full = criar_janelas(dados_val_train_scaled, janela)

    if len(X_val_train_full) < n_splits_cv:
         print(f"Aviso: Não há dados suficientes ({len(X_val_train_full)} amostras) para {n_splits_cv} splits em {ticker}, Janela {janela}. Pulando.")
         return

    # 2. Otimização de Hiperparâmetros (Keras Tuner)
    print("Iniciando busca de hiperparâmetros...")
    tuner = kt.RandomSearch(
        lambda hp: build_model(hp, input_shape=(janela, 1)),
        objective='val_loss',
        max_trials=max_trials_tuner,
        executions_per_trial=1, # Rodar cada trial uma vez
        directory='keras_tuner_cv',
        project_name=f'LSTM_{ticker}_Janela_{janela}',
        overwrite=True
    )

    # Usar EarlyStopping dentro do tuner search para eficiência
    early_stopping_tuner = EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

    # Dividir os dados de treino/validação (2020-2022) para o tuner
    # Usaremos uma divisão simples aqui, mas garantindo a ordem temporal
    split_index = int(len(X_val_train_full) * 0.8) # Ex: 80% treino, 20% validação para o tuner
    X_tuner_train, y_tuner_train = X_val_train_full[:split_index], y_val_train_full[:split_index]
    X_tuner_val, y_tuner_val = X_val_train_full[split_index:], y_val_train_full[split_index:]

    if len(X_tuner_val) == 0:
        print(f"Aviso: Não há dados de validação suficientes para o tuner em {ticker}, Janela {janela} após split. Usando treino completo para busca (menos ideal).")
        tuner.search(X_val_train_full, y_val_train_full, epochs=epochs_tuner,
                     callbacks=[early_stopping_tuner], verbose=1)
    else:
        tuner.search(X_tuner_train, y_tuner_train, epochs=epochs_tuner,
                     validation_data=(X_tuner_val, y_tuner_val),
                     callbacks=[early_stopping_tuner], verbose=1)

    best_hps = tuner.get_best_hyperparameters(1)[0]
    print("\nMelhores Hiperparâmetros encontrados:")
    print(best_hps.values)

    # 3. Avaliação Cruzada dos Melhores Hiperparâmetros (TimeSeriesSplit no período 2020-2022)
    print(f"\nIniciando TimeSeriesSplit CV ({n_splits_cv} folds) para avaliar os melhores HPs...")
    tscv = TimeSeriesSplit(n_splits=n_splits_cv)
    cv_metrics = {'mae': [], 'mse': [], 'rmse': [], 'r2': []}

    fold = 0
    for train_index, val_index in tscv.split(X_val_train_full):
        fold += 1
        print(f"  Fold {fold}/{n_splits_cv}...")
        X_train_fold, X_val_fold = X_val_train_full[train_index], X_val_train_full[val_index]
        y_train_fold, y_val_fold = y_val_train_full[train_index], y_val_train_full[val_index]

        # Construir e treinar modelo com os melhores HPs neste fold
        model_cv = tuner.hypermodel.build(best_hps)
        # Early stopping também no treino do fold CV
        early_stopping_cv = EarlyStopping(monitor='loss', patience=10, verbose=0) # Monitorar treino loss aqui
        model_cv.fit(X_train_fold, y_train_fold, epochs=epochs_final, # Usar epochs_final aqui tbm
                     batch_size=32, # Definir batch size
                     callbacks=[early_stopping_cv],
                     verbose=0) # Menos verbose dentro do CV

        # Avaliar no conjunto de validação do fold
        previsoes_val_fold_scaled = model_cv.predict(X_val_fold)
        previsoes_val_fold = scaler.inverse_transform(previsoes_val_fold_scaled)
        reais_val_fold = scaler.inverse_transform(y_val_fold.reshape(-1, 1))

        # Calcular métricas do fold
        cv_metrics['mae'].append(mean_absolute_error(reais_val_fold, previsoes_val_fold))
        cv_metrics['mse'].append(mean_squared_error(reais_val_fold, previsoes_val_fold))
        cv_metrics['rmse'].append(np.sqrt(cv_metrics['mse'][-1]))
        cv_metrics['r2'].append(r2_score(reais_val_fold, previsoes_val_fold))
        print(f"    Fold {fold} MAE: {cv_metrics['mae'][-1]:.4f}, RMSE: {cv_metrics['rmse'][-1]:.4f}, R2: {cv_metrics['r2'][-1]:.4f}")


    # Calcular métricas médias da validação cruzada
    avg_cv_mae = np.mean(cv_metrics['mae'])
    avg_cv_mse = np.mean(cv_metrics['mse'])
    avg_cv_rmse = np.mean(cv_metrics['rmse'])
    avg_cv_r2 = np.mean(cv_metrics['r2'])
    print("\nResultados Médios da Validação Cruzada (TimeSeriesSplit):")
    print(f"  Avg MAE: {avg_cv_mae:.4f}")
    print(f"  Avg MSE: {avg_cv_mse:.4f}")
    print(f"  Avg RMSE: {avg_cv_rmse:.4f}")
    print(f"  Avg R2: {avg_cv_r2:.4f}")

    # 4. Treinamento Final (com melhores HPs, usando TODOS os dados de 2020-2022)
    print("\nTreinando modelo final com melhores HPs em todos os dados de 2020-2022...")
    model_final = tuner.hypermodel.build(best_hps)
    early_stopping_final = EarlyStopping(monitor='loss', patience=15, verbose=1, restore_best_weights=True) # Monitorar loss, patience maior
    model_final.fit(X_val_train_full, y_val_train_full, epochs=epochs_final,
                    batch_size=32, # Usar batch size
                    callbacks=[early_stopping_final],
                    verbose=1)

    # 5. Avaliação Final no Conjunto de Teste (2023 - Hold-Out)
    print("\nAvaliando modelo final no conjunto de teste (2023)...")
    df_test_final = dados_ticker['test_final'].copy()
    if df_test_final.isnull().any().any():
        print(f"Aviso: Dados nulos encontrados em df_test_final para {ticker}. Preenchendo com ffill/bfill.")
        df_test_final.fillna(method='ffill', inplace=True)
        df_test_final.fillna(method='bfill', inplace=True)

    # Precisamos escalar os dados de teste USANDO O MESMO SCALER ajustado nos dados de treino
    dados_test_scaled = scaler.transform(df_test_final)
    X_test_final, y_test_final = criar_janelas(dados_test_scaled, janela)

    if len(X_test_final) == 0:
        print(f"Erro: Não foi possível criar janelas para o conjunto de teste final de {ticker}, Janela {janela}. Verifique as datas e o tamanho da janela.")
        return

    previsoes_test_scaled = model_final.predict(X_test_final)
    previsoes_test_final = scaler.inverse_transform(previsoes_test_scaled)
    reais_test_final = scaler.inverse_transform(y_test_final.reshape(-1, 1))

    # Calcular métricas do teste final
    mae_test = mean_absolute_error(reais_test_final, previsoes_test_final)
    mse_test = mean_squared_error(reais_test_final, previsoes_test_final)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(reais_test_final, previsoes_test_final)
    print("\nResultados da Avaliação Final (Teste 2023):")
    print(f"  MAE Teste: {mae_test:.4f}")
    print(f"  MSE Teste: {mse_test:.4f}")
    print(f"  RMSE Teste: {rmse_test:.4f}")
    print(f"  R² Teste: {r2_test:.4f}")

    # 6. Salvar Resultados
    print("\nSalvando resultados...")
    # Salvar métricas (CV e Teste)
    metrics_df = pd.DataFrame([{
        'Ticker': ticker,
        'Janela': janela,
        'CV Avg MAE': avg_cv_mae,
        'CV Avg MSE': avg_cv_mse,
        'CV Avg RMSE': avg_cv_rmse,
        'CV Avg R2': avg_cv_r2,
        'Teste MAE': mae_test,
        'Teste MSE': mse_test,
        'Teste RMSE': rmse_test,
        'Teste R2': r2_test
    }])
    metrics_df.to_csv(metrics_path, index=False)
    print(f"Métricas salvas em {metrics_path}")

    # Salvar hiperparâmetros
    hiperparametros_df = pd.DataFrame([best_hps.values])
    hiperparametros_df.to_csv(hiperparametros_path, index=False)
    print(f"Hiperparâmetros salvos em {hiperparametros_path}")

    # Salvar previsões do teste final
    # Ajustar as datas para corresponder às previsões (começam 'janela' dias depois do início do teste)
    datas_teste_final = df_test_final.index[janela:]
    # Verificar se o número de datas corresponde ao número de previsões
    if len(datas_teste_final) == len(reais_test_final):
        previsoes_df = pd.DataFrame({
            'Data': datas_teste_final,
            'Preço Real': reais_test_final.flatten(),
            'Preço Previsto': previsoes_test_final.flatten()
        })
        previsoes_df.to_csv(previsoes_path, index=False)
        print(f"Previsões do teste salvas em {previsoes_path}")

        # Gerar gráfico de preços reais vs previstos (Teste Final)
        plt.figure(figsize=(14, 7))
        plt.plot(previsoes_df['Data'], previsoes_df['Preço Real'], label='Real (Teste 2023)', color='blue', linewidth=1.5)
        plt.plot(previsoes_df['Data'], previsoes_df['Preço Previsto'], label='Previsto (Teste 2023)', color='orange', linestyle='--', linewidth=1.5)
        plt.title(f'Preços Reais vs Previstos (Teste Final) - {ticker} (Janela {janela})')
        plt.xlabel('Data')
        plt.ylabel('Preço')
        plt.legend()
        plt.grid(alpha=0.4)
        plt.tight_layout()
        plt.savefig(grafico_path)
        print(f"Gráfico do teste salvo em {grafico_path}")
        # plt.show() # Descomentar se quiser exibir interativamente
        plt.close() # Fechar a figura para liberar memória
    else:
        print(f"Erro: Discrepância entre número de datas ({len(datas_teste_final)}) e previsões ({len(reais_test_final)}) para {ticker}, Janela {janela}. Gráfico/Previsões não salvos.")

    print(f"--- Experimento Concluído: {ticker}, Janela {janela} ---")


# --- Loop Principal de Experimentos MODIFICADO ---
def rodar_experimentos_cv():
    for ticker in tickers:
        if ticker not in dados_por_ticker:
             print(f"Dados para {ticker} não foram carregados. Pulando ticker.")
             continue
        dados_ticker = dados_por_ticker[ticker]
        for janela in range(janela_min, janela_max + 1): # Usar range correto
            try:
                # Passar os dados específicos do ticker para a função
                rodar_experimento_especifico(ticker, janela, dados_ticker)
            except Exception as e:
                print(f"Erro CRÍTICO ao rodar experimento {ticker}, Janela {janela}: {e}")
                # Considerar logar o traceback completo para depuração
                import traceback
                traceback.print_exc()

# Rodar os experimentos com a nova estrutura
rodar_experimentos_cv()

print("\n--- TODOS OS EXPERIMENTOS CONCLUÍDOS ---")

Trial 10 Complete [00h 00m 17s]
val_loss: 0.00041789491660892963

Best val_loss So Far: 0.00041789491660892963
Total elapsed time: 00h 02m 15s

Melhores Hiperparâmetros encontrados:
{'num_lstm_layers': 1, 'units_lstm_0': 96, 'dropout_lstm_0': 0.2, 'num_dense_layers': 2, 'learning_rate': 0.001, 'units_lstm_1': 96, 'dropout_lstm_1': 0.1, 'units_lstm_2': 64, 'dropout_lstm_2': 0.1, 'units_dense_0': 32, 'dropout_dense_0': 0.2, 'units_dense_1': 48, 'dropout_dense_1': 0.2}

Iniciando TimeSeriesSplit CV (5 folds) para avaliar os melhores HPs...
  Fold 1/5...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
    Fold 1 MAE: 2.6686, RMSE: 3.1220, R2: 0.5291
  Fold 2/5...


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step
    Fold 2 MAE: 1.2610, RMSE: 1.5364, R2: 0.9127
  Fold 3/5...


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
    Fold 3 MAE: 0.8928, RMSE: 1.1722, R2: 0.8926
  Fold 4/5...


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
    Fold 4 MAE: 0.9730, RMSE: 1.2098, R2: 0.8847
  Fold 5/5...


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
    Fold 5 MAE: 1.4121, RMSE: 1.6572, R2: 0.8048

Resultados Médios da Validação Cruzada (TimeSeriesSplit):
  Avg MAE: 1.4415
  Avg MSE: 3.5383
  Avg RMSE: 1.7395
  Avg R2: 0.8048

Treinando modelo final com melhores HPs em todos os dados de 2020-2022...
Epoch 1/100


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2519
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0339 
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0230 
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0157 
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0125 
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0122 
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0091 
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0072 
Epoch 9/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0077 
Epoch 10/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0082
Epoch 11/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - loss: 0.0071
Epoch 12/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0075 
Epoch 13/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0064 
Epoch 14/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0063 
Epoch 15/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0063 
Ep